# Regresión Lineal Ordinaria (LSS)

En esta sección trabajaremos con un dataset conocido como House Sales in King County, USA, presentado
en la plataforma de Kaggle [4], el cual es un gran dataset para evaluar simples modelos de regresión. Los
registros contienen distintas características asociadas a las ventas de casas en la localidad King County, entre
mayo de 2014 y mayo de 2015, las cuales vienen descritas en el dataset, como la cantidad de habitaciones,
cantidad de baños, número de pisos, etc. Donde una de las variables a estudiar corresponde al precio en el
cual se vendió la casa.

  * Construya un dataframe con los datos a analizar descargándolos desde la plataforma como se indicó.
Explique por qué se realiza la línea 4.


In [3]:
import pandas as pd
import numpy as np
df = pd.read_csv("kc_house_data.csv")
df.drop(['id','date','zipcode',],axis=1,inplace=True)
df.head()

ImportError: No module named 'pandas'